**IMPORTANT LINKS**

[Descriptive article](https://towardsdatascience.com/fake-face-generator-using-dcgan-model-ae9322ccfd65)

[General overview article](https://www.lyrn.ai/2018/12/26/a-style-based-generator-architecture-for-generative-adversarial-networks/)

**PLASE NOTE THAT THIS CODE WILL FAIL TO RUN SUCCESFULLY ON THE CLOUD, AND MUST BE RUN ON A LOCAL MACHINE**

This is currently simply here just to be viewed.

**NOTE**

CODE IS IN FINAL STATE
NOT TO FURTHER BE EDITED EXCEPT MODIFICATIONS FOR FINAL SUBMISSION

NICE TEAMWORK!

In [0]:
# IMPORTS
import os
import time
import tensorflow as tf
import numpy as np 
from glob import glob
import datetime
import random
from PIL import Image

In [0]:
def generator(z, outputChannelDim, training):
	with tf.variable_scope("generator", reuse= not training):
		# 8 by 8 by 1024
		fullyConnected = tf.layers.dense(z, 8*8*1024)
		fullyConnected = tf.reshape(fullyConnected, (-1, 8, 8, 1024))
		fullyConnected = tf.nn.leaky_relu(fullyConnected)

		# 16 by 16 by 512
		transConv1 = tf.layers.conv2d_transpose(inputs = fullyConnected, filters=512, kernel_size=[5,5], strides=[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name = "transConv1")
		batchTransConv1 = tf.layers.batch_normalization(inputs = transConv1, training=training, epsilon=EPSILON, name="batchTransConv1")
		transConv1Out = tf.nn.leaky_relu(batchTransConv1, name="transConv1Out")

		# 32 by 32 by 256
		transConv2 = tf.layers.conv2d_transpose(inputs=transConv1Out, filters=256, kernel_size=[5,5], strides=[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="transConv2")
		batchTransConv2 = tf.layers.batch_normalization(inputs=transConv2, training=training, epsilon=EPSILON, name="batchTransConv2")
		transConv2Out = tf.nn.leaky_relu(batchTransConv2, name="transConv2Out")

		# 64 by 64 by 128
		transConv3 = tf.layers.conv2d_transpose(inputs=transConv2Out, filters=128, kernel_size=[5,5], strides=[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="transConv3")
		batchTransConv3 = tf.layers.batch_normalization(inputs=transConv3, training=training, epsilon=EPSILON, name="batchTransConv3")
		transConv3Out = tf.nn.leaky_relu(batchTransConv3, name="transConv3Out")

		# 128 by 128 by 64
		transConv4 = tf.layers.conv2d_transpose(inputs=transConv3Out, filters=64, kernel_size=[5,5], strides=[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="transConv4")
		batchTransConv4 = tf.layers.batch_normalization(inputs=transConv4, training=training, epsilon=EPSILON, name="batchTransConv4")
		transConv4Out = tf.nn.leaky_relu(batchTransConv4, name="transConv4Out")

		# 128 by 128 by 3
		logits = tf.layers.conv2d_transpose(inputs=transConv4Out, filters=3, kernel_size=[5,5], strides=[1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="logits")
		output = tf.tanh(logits, name="output")

		return output


In [0]:
def discriminator(x, reuse):
    with tf.variable_scope("discriminator", reuse=reuse):
        # 64 by 64 by 64
        conv1 = tf.layers.conv2d(inputs=x, filters=64, kernel_size=[5,5], strides=[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="conv1")
        batchNorm1 = tf.layers.batch_normalization(conv1, training=True, epsilon=EPSILON, name="batchNorm1")
        conv1Out = tf.nn.leaky_relu(batchNorm1, name="conv1Out")

        # 32 by 32 by 128
        conv2 = tf.layers.conv2d(inputs=conv1Out, filters=128, kernel_size=[5,5], strides=[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="conv2")
        batchNorm2 = tf.layers.batch_normalization(conv2, training=True, epsilon=EPSILON, name="batchConv2")
        conv2Out = tf.nn.leaky_relu(batchNorm2, name="conv2Out")

        # 16 by 16 by 256
        conv3 = tf.layers.conv2d(inputs=conv2Out, filters=256, kernel_size=[5,5], strides=[2,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="conv3")
        batchNorm3 = tf.layers.batch_normalization(conv3, training=True, epsilon=EPSILON, name="batchNorm3")
        conv3Out = tf.nn.leaky_relu(batchNorm3, name="conv3Out")

        # 16 by 16 by 512
        conv4 = tf.layers.conv2d(inputs=conv3Out, filters=512, kernel_size=[5,5], strides=[1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="conv4")
        batchNorm4 = tf.layers.batch_normalization(conv4, training=True, epsilon=EPSILON, name="batchNorm4")
        conv4Out = tf.nn.leaky_relu(batchNorm4, name="conv4Out")

        # 8 by 8 by 1024
        conv5 = tf.layers.conv2d(inputs=conv4Out, filters=1024, kernel_size=[5,5], strides=[1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(stddev=WEIGHT_INIT_STDDEV), name="conv5")
        batchNorm5 = tf.layers.batch_normalization(conv5, training=True, epsilon=EPSILON, name="batchNorm5")
        conv5Out = tf.nn.leaky_relu(batchNorm4, name="conv5Out")

        flatten = tf.reshape(conv5Out, (-1, 8*8*1024))
        logits = tf.layers.dense(inputs=flatten, units=1, activation=None)
        output = tf.sigmoid(logits)
        return output, logits

In [0]:
def modelLoss(inputReal, inputZ, outputChannelDim):
    gModel = generator(inputZ, outputChannelDim, True)

    noisyInputReal = inputReal + tf.random_normal(shape=tf.shape(inputReal), mean=0.0, stddev=random.uniform(0.0, 0.1), dtype=tf.float32)

    dModelReal, dLogitsReal = discriminator(noisyInputReal, reuse=False)
    dModelFake, dLogitsFake = discriminator(gModel, reuse=True)

    dLossReal = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dLogitsReal, labels=tf.ones_like(dModelReal)*random.uniform(0.9, 1.0)))
    dLossFake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dLogitsFake, labels=tf.zeros_like(dModelFake)))

    dLoss = tf.reduce_mean(0.5 * (dLossReal + dLossFake))
    gLoss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=dLogitsFake, labels=tf.ones_like(dModelFake)))
    
    return dLoss, gLoss

In [0]:
def modelOptimizers(dLoss, gLoss):
    tVars = tf.trainable_variables()
    gVars = [var for var in tVars if var.name.startswith("generator")]
    dVars = [var for var in tVars if var.name.startswith("discriminator")]

    updateOps = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    genUpdates = [op for op in update_ops if op.name.startswith("generator") or op.name.startswith("discriminator")]

    with tf.control_dependencies(genUpdates):
        dTrainOpt = tf.train.AdamOptimizer(learning_rate=LEARN_D, beta1=BETA1).minimize(dLoss, var_list=dVars)
        gTrainOpt = tf.train.AdamOptimizer(learning_rate=LEARN_G, beta1=BETA1).minimize(gLoss, var_list=gVars)
        
    return dTrainOpt, gTrainOpt

In [0]:
def modelInputs(realDim, zDim):
    inputsReal = tf.placeholder(tf.float32, (None, *realDim), name="inputsReal")
    inputsZ = tf.placeholder(tf.float32, (None, zDim), name="inputZ")
    learningRateG = tf.placeholder(tf.float32, name="learningRateG")
    learningRateD = tf.placeholder(tf.float32, name="learningRateD")
    return inputsReal, inputsZ, learningRateG, learningRateD

In [0]:
# function deprecated, does not work on local machines; therefore has been omitted
# FUNCTION TO BE ALTERED TO ENABLE IMAGE SAVING ON LOCAL PC
"""def showSamples(sampleImages, name, epoch):
    figure, axes = plt.subplots(1, len(sampleImages), figsize=(IMG_SIZE, IMG_SIZE))
    for index, axis in enumerate(axes):
        axis.axis('off')
        imageArray = sampleImages[index]
        axis.imshow(imageArray)
        image = Image.fromarray(imageArray)
        image.save(name+"_"+str(epoch)+"_"+str(index)+".png")
    plt.savefig(name+"_"+str(epoch)+".png", bbox_inches='tight', pad_inches=0)
    plt.show()
    plt.close() """

In [0]:
def test(sess, inputZ, outChannelDim, epoch):
    exampleZ = np.random.uniform(-1, 1, size=[SAMPLES_TO_SHOW, inputZ.get_shape().as_list()[-1]])
    samples=sess.run(generator(inputZ, outChannelDim, False), feed_dict={inputZ: exampleZ})
    sampleImages = [((sample + 1.0) * 127.5).astype(np.uint8) for sample in samples]
    #show_samples(sampleImages, OUTPUT_DIR + "samples", epoch) -- THIS LINE TO BE OMITTED

In [0]:
def summEpoch(epoch, sess, dLosses, gLosses, inputZ, dataShape, saver):
    print("\nEpoch {}/{}".format(epoch, EPOCHS), "\nD Loss: {:.5f}".format(np.mean(dLosses[-MINIBATCH_SIZE:])), "\nG Loss: {:.5f}".format(np.mean(gLosses[-MINIBATCH_SIZE:])))
    # FOLLOWING LINES TO BE OMITTED
    """
	fig, ax = plt.subplots()
    plt.plot(dLosses, label='Discriminator', alph=0.6)
    plt.plot(gLosses, label='Generator', alpha=0.6)
    plt.title("Losses")
    plt.legend()
    plt.savefig(OUTPUT_DIR + "losses_" + str(epoch) + ".png")
    plt.show()
    plt.close()
	"""
    saver.save(sess, OUTPUT_DIR + "model_" + str(epoch) + ".ckpt")
    test(sess, inputZ, dataShape[3], epoch)

In [0]:
def getBatch(dataset):
	files = random.sample(dataset, BATCH_SIZE)
	batch = []
	for file in files:
		if random.choice([True, False]):
			batch.append(np.asarray(Image.open(file).transpose(Image.FLIP_LEFT_RIGHT)))
		else:
			batch.append(np.asarray(Image.open(file)))
	batch = np.asarray(batch)
	normalizedBatch = (batch / 127.5) - 1.0
	return normalizedBatch, files

In [0]:
def train(dataShape, epoch, checkpointPath):
    inputImages, inputZ, learningRateG, learningRateD = modelInputs(dataShape[1:], NOISE_SIZE)
    dLoss, gLoss = modelLoss(inputImages, inputZ, dataShape[3])
    dOpt, gOpt = modelOptimizers(dLoss, gLoss)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver()
        if checkpointPath is not None:
            saver.restore(sess, checkpointPath)

        iteration = 0
        dLosses = []
        gLosses = []

        for epoch in range(EPOCH, EPOCHS):
            epoch += 1
            epochDataset = DATASET.copy()

            for i in range(MINIBATCH_SIZE):
                iterationStartTime = time.time()
                iteration += 1
                batchImages, usedFiles = getBatch(epochDataset)
                [epochDataset.remove(file) for file in usedFiles]

                batchZ = np.random.uniform(-1, 1, size=(BATCH_SIZE, NOISE_SIZE))
                _ = sess.run(dOpt, feed_dict={inputImages: batchImages, inputZ: batchZ, learningRateD: LEARN_D})
                _ = sess.run(gOpt, feed_dict={inputImages: batchImages, inputZ: batchZ, learningRateG: LEARN_G})
                dLosses.append(dLoss.eval({inputZ: batchZ, inputImages: batchImages}))
                gLosses.append(gLoss.eval({inputZ: batchZ}))

                elapsedTime = round(time.time()-iterationStartTime, 3)
                remainingFiles = len(epochDataset)
                print("\rEpoch: " + str(epoch) +
                      ", iteration: " + str(iteration) + 
                      ", d_loss: " + str(round(dLosses[-1], 3)) +
                      ", g_loss: " + str(round(gLosses[-1], 3)) +
                      ", duration: " + str(elapsedTime) + 
                      ", minutes remaining: " + str(round(remainingFiles/BATCH_SIZE*elapsedTime/60, 1)) +
                      ", remaining files in batch: " + str(remainingFiles))
                summEpoch(epoch, sess, dLosses, gLosses, inputZ, dataShape, saver)

In [0]:
# HYPERPARAMS
IMAGE_SIZE = 128
NOISE_SIZE = 100
LEARN_D = 0.00004
LEARN_G = 0.0002
BATCH_SIZE = 64
EPOCH = 0 # not 0 if resuming session
EPOCHS = 8
BETA1 = 0.5
WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.00005
SAMPLES_TO_SHOW = 5

In [0]:
BASE_PATH="../input/"
DATASET_LIST_PATH = BASE_PATH + "100k.txt"
INPUT_DATA_DIR = BASE_PATH + "100k/100k/"
OUTPUT_DIR = "./"
DATASET = [INPUT_DATA_DIR + str(line).rstrip() for line in open(DATASET_LIST_PATH, "r")]
DATASET_SIZE = len(DATASET)
MINIBATCH_SIZE = DATASET_SIZE // BATCH_SIZE

In [0]:
# to resume training:
# MODEL_PATH = BASE_PATH + "models/" + "model_" + str(EPOCH) + ".ckpt"

In [0]:
# Execution!
with tf.Graph().as_default():
    train(dataShape=(DATASET_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3), epoch=EPOCH, checkpointPath=None)